In [1]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd 
import requests
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
import os
from googleapiclient.discovery import build
import configparser

#### Web Page Elements 
Videos_List : style-scope ytd-rich-item-renderer

title --  //*[@id="video-title-link"]

views -- //*[@id="metadata-line"]/span[1]

day_posted -- //*[@id="metadata-line"]/span[2]
video_link -- //*[@id="thumbnail"]


In [ ]:
# PATH = r'chromedriver.exe'
# service = Service(executable_path=PATH)
# wd = webdriver.Chrome(service=service)
# url = "https://www.youtube.com/@krishnaik06/videos"
# wd.get(url)
# videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        
        
# for video in videos:
#     title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").get_attribute('href')
#     print(title)


## Scrolling Till 50 Links are reached


In [91]:
def scroll_to_end(wd):
    wd.execute_script("window.scrollTo(0, 1000);")
    time.sleep(5)

def fetch_video_details(url, max_videos=50):
    PATH = r'chromedriver.exe'
    service = Service(executable_path=PATH)
    wd = webdriver.Chrome(service=service)
    wd.get(url)
    
    video_details = []
    while len(video_details) < max_videos:
        videos = wd.find_elements(By.CLASS_NAME, "style-scope ytd-rich-item-renderer")
        scroll_to_end(wd)
        
        for video in videos:
            title = video.find_element(By.XPATH, ".//*[@id='video-title-link']").text
            view = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[1]").text
            date = video.find_element(By.XPATH, ".//*[@id='metadata-line']/span[2]").text
            link = video.find_element(By.XPATH, ".//*[@id='video-title-link']").get_attribute('href') 
            # duration = video.find_element(By.CLASS_NAME, "style-scope ytd-thumbnail-overlay-time-status-renderer").text
        

            details = {
                'title': title,
                'views': view,
                'date': date,
                'yt_link':link
            }
            
            video_details.append(details)
            if len(video_details) >= max_videos:
                break
    
    wd.quit()
    return pd.DataFrame(video_details)

url = "https://www.youtube.com/@krishnaik06/videos"
max_videos = 50
video_details_df = fetch_video_details(url, max_videos)


channel_ids = ["UCNU_lfiiWBdtULKOw6X0Dig","UCnz-ZXXER4jOvuED5trXfEA","UCx6vgn7wVgo4ioAkzPJf71A","UC8butISFwT-Wl7EV0hUK0BQ"]

## Likes, Dislikes, Description, Title, Comment 

In [4]:
# USE API 
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config["USER"]['API_KEY']
youtube = build('youtube','v3',developerKey=api_key)


In [13]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(part="snippet,contentDetails,statistics",id=channel_id)
    response = request.execute()
    data = {
        'Channel_Name': response['items'][0]['snippet']['title'],
        'Subscribers': response['items'][0]['statistics']['subscriberCount'],
        'Views': response['items'][0]['statistics']['viewCount'],
        'Total_Videos':response['items'][0]['statistics']['videoCount'],
        'All_videos':response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    }

    return data 

In [14]:
get_channel_stats(youtube,"UCNU_lfiiWBdtULKOw6X0Dig")

{'Channel_Name': 'Krish Naik',
 'Subscribers': '802000',
 'Views': '80541929',
 'Total_Videos': '1706',
 'All_videos': 'UUNU_lfiiWBdtULKOw6X0Dig'}